# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a pandas DataFrame.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [2]:
import requests as r
print(r.certs.where())

C:\PythonBook\Hands-On-Data-Analysis-with-Pandas\book_env\lib\site-packages\certifi\cacert.pem


In [1]:
import datetime
import pandas as pd
import requests

yesterday = datetime.date.today() - datetime.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format' : 'geojson',
    'starttime' : yesterday - datetime.timedelta(days=26),
    'endtime' : yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
print(response.status_code)

SSLError: HTTPSConnectionPool(host='earthquake.usgs.gov', port=443): Max retries exceeded with url: /fdsnws/event/1/query?format=geojson&starttime=2022-05-01&endtime=2022-05-27 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON, we can extract it from the response with the JSON method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [ ]:
earthquake_json = response.json()
earthquake_json.keys()

In [ ]:
earthquake_json['metadata']

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.5,
  'place': '123km WNW of Arctic Village, Alaska',
  'time': 1539387977830,
  'updated': 1539993599455,
  'tz': -540,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak20279829',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak20279829&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 35,
  'net': 'ak',
  'code': '20279829',
  'ids': ',ak20279829,',
  'sources': ',ak,',
  'types': ',geoserve,origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.46,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.5 - 123km WNW of Arctic Village, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-148.3878, 68.4597, 17.8]},
 'id': 'ak20279829'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [data['properties'] for data in earthquake_json['features']]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,None,NaN,20279829,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,",ak20279829,",1.50,ml,...,",ak,",reviewed,1539387977830,"M 1.5 - 123km WNW of Arctic Village, Alaska",0,earthquake,",geoserve,origin,phase-data,",-540.0,1539993599455,https://earthquake.usgs.gov/earthquakes/eventp...
1,None,NaN,20279827,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,",ak20279827,",1.80,ml,...,",ak,",reviewed,1539387643408,"M 1.8 - 72km NNW of Arctic Village, Alaska",0,earthquake,",geoserve,origin,phase-data,",-540.0,1539993598904,https://earthquake.usgs.gov/earthquakes/eventp...
2,None,NaN,61437241,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.17560,NaN,335.0,",uw61437241,",1.49,ml,...,",uw,",reviewed,1539387421260,"M 1.5 Explosion - 2km N of Hockinson, Washington",0,explosion,",geoserve,origin,phase-data,",-480.0,1539631055160,https://earthquake.usgs.gov/earthquakes/eventp...
3,None,NaN,20279823,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,",ak20279823,",2.20,ml,...,",ak,",reviewed,1539387411152,"M 2.2 - 78km SSW of Kaktovik, Alaska",0,earthquake,",geoserve,origin,phase-data,",-540.0,1539993598323,https://earthquake.usgs.gov/earthquakes/eventp...
4,None,NaN,37387882,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.01668,NaN,45.0,",ci37387882,",1.14,ml,...,",ci,",reviewed,1539387294800,"M 1.1 - 9km NE of Aguanga, CA",0,earthquake,",focal-mechanism,geoserve,nearby-cities,origin...",-480.0,1539613956040,https://earthquake.usgs.gov/earthquakes/eventp...


### (Optional) Write Data to CSV

In [7]:
pd.DataFrame(earthquake_properties_data).to_csv('earthquakes.csv', index=False)